In [ ]:
from PIL import Image
import os

In [ ]:
def resize_image(filename, input_folder, output_folder, target_size_kb=500):
    """Resize image to meet target size by adjusting resolution, not quality."""
    filepath = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)

    try:
        image = Image.open(filepath)
        image = image.convert("RGB")  # Convert to RGB (ensures compatibility)

        # Save once at full quality to check if resizing is needed
        image.save(output_path, "JPEG", quality=100, optimize=True)
        if os.path.getsize(output_path) / 1024 <= target_size_kb:
            print(f"✔ {filename}: Already under {target_size_kb}KB")
            return

        # Start resizing process
        width, height = image.size
        scale_factor = 0.99  # Start with a 90% scale reduction

        while os.path.getsize(output_path) / 1024 > target_size_kb:
            new_size = (int(width * scale_factor), int(height * scale_factor))
            resized_image = image.resize(new_size, Image.LANCZOS)
            resized_image.save(output_path, "JPEG", quality=100, optimize=True)

            width, height = new_size  # Update new size for next iteration
            scale_factor -= 0.01  # Reduce size in steps of 5%
            if min(width, height) < 300:  # Avoid excessive downsizing
                break

        print(f"✔ {filename}: Resized to {os.path.getsize(output_path) / 1024:.2f} KB")

    except Exception as e:
        print(f"❌ Error processing {filename}: {e}")

def resize_images(input_folder, output_folder, target_size_kb=500):
    """Process all images in the input folder and resize them to the target size."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(("jpg", "jpeg", "png")):
            resize_image(filename, input_folder, output_folder, target_size_kb)

In [ ]:
# define folder where the original images can be found
input_folder = "/original/"
output_folder = "/compressed/"

resize_images(input_folder, output_folder, target_size_kb=500)